<em style="text-align:center">Copyright Iván Pinar Domínguez</em>

#  Enrutamiento con LLM Router Chain

## Importar librerías iniciales e instancia de modelo de chat

In [1]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SequentialChain, LLMChain #importamos el SequentialChain que es el modelo completo
f = open('../OpenAI_key.txt')
api_key = f.read()
llm = ChatOpenAI(openai_api_key=api_key)

### Plantillas de enrutamiento

In [2]:
#Template (prompt) para soporte básico a clientes de coches
plantilla_soporte_basico_cliente = '''Eres una persona que asiste a los clientes de automóviles con preguntas básicas que pueden
necesitar en su día a día y que explica los conceptos de una manera que sea simple de entender. Asume que no tienen conocimiento
previo. Esta es la pregunta del usuario/n{input}'''

In [3]:
#Template (prompt) para soporte avanzados a nuestros expertos en mecánica
plantilla_soporte_avanzado_mecánico = '''Eres un experto en mecánica que explicas consultas avanzadas a los mecánicos
de la plantilla. Puedes asumir que cualquier que está preguntando tiene conocimientos avanzados de mecánica. 
Esta es la pregunta del usuario/n{input}'''

In [156]:
#Se pueden añadir todas las plantillas que necesitemos

### Prompts a enrutar

In [4]:
#Debemos crear una lista de diccionarios, cada diccionario contiene su nombre, la descripción (en base a la cual el enrutador
#hará su trabajo) y el prompt a usar en cada caso
prompt_infos = [
    {'name':'mecánica básica','description': 'Responde preguntas básicas de mecánicas a clientes',
     'prompt_template':plantilla_soporte_basico_cliente},
    {'name':'mecánica avanzada','description': 'Responde preguntas avanzadas de mecánica a expertos con conocimiento previo',
     'prompt_template':plantilla_soporte_avanzado_mecánico},
    
]

In [6]:
#TODO LO QUE VIENE A CONTINUACIÓN ES AGNÓSTICO DEL CASO DE USO Y LO PUEDES UTILIZAR PARA TODOS LOS CASOS DE USO

### ConversationChain

In [5]:
from langchain.chains import LLMChain

In [7]:
#Creamos un diccionario de objetos LLMChain con las posibles cadenas destino
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [8]:
destination_chains

{'mecánica básica': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='Eres una persona que asiste a los clientes de automóviles con preguntas básicas que pueden\nnecesitar en su día a día y que explica los conceptos de una manera que sea simple de entender. Asume que no tienen conocimiento\nprevio. Esta es la pregunta del usuario/n{input}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001E47A072B10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001E47A073850>, openai_api_key=SecretStr('**********'), openai_proxy='')),
 'mecánica avanzada': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='Eres un experto en mecánica que explicas consultas avanzadas a los mecánicos\nde la plantilla. Puedes asumir q

In [9]:
#Creamos el prompt y cadena por defecto puesto que son argumento obligatorios que usaremos posteriormente
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

### Multi Routing Template

In [10]:
#Importamos una plantilla que podremos formatear su parámetro {destinations} que tendrá cada nombre y descripción de la información de prompts
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [12]:
print(MULTI_PROMPT_ROUTER_TEMPLATE) #El parámetro importante es {destinations}, debemos formatearlo en tipo string

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

### Destinos de Routing

In [13]:
#Creamos una string global con todos los destinos de routing usando el nombre y descripción de "prompt_infos"
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [14]:
destinations_str

'mecánica básica: Responde preguntas básicas de mecánicas a clientes\nmecánica avanzada: Responde preguntas avanzadas de mecánica a expertos con conocimiento previo'

### Router Prompt

In [15]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [16]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str #Formateamos la plantilla con nuestros destinos en la string destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), #Para transformar el objeto JSON parseándolo a una string
)

In [17]:
print(router_template) #verificamos que se ha formateado correctamente

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
mecánica básica: Responde preguntas básicas de mecánicas a clientes
mecánica avanzada: Responde pregu

### Routing Chain Call

In [18]:
from langchain.chains.router import MultiPromptChain

In [19]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [21]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, #El objeto con los posibles LLMChain que creamos al inicio
                         default_chain=default_chain, verbose=True #Indicamos el LLMChain por defecto (obligatorio)
                        )

In [22]:
chain.invoke("¿Cómo cambio el aceite de mi coche?")



> Entering new MultiPromptChain chain...
mecánica básica: {'input': '¿Cómo cambio el aceite de mi coche?'}
> Finished chain.


{'input': '¿Cómo cambio el aceite de mi coche?',
 'text': 'Cambiar el aceite de tu coche es una tarea importante para mantener el motor en buen estado. Aquí te explico de manera sencilla cómo hacerlo:\n\n1. Reúne los materiales necesarios: aceite nuevo, un filtro de aceite nuevo, una llave para quitar el tapón de drenaje y un recipiente para recoger el aceite usado.\n\n2. Calienta el motor: es importante que el motor esté caliente para que el aceite salga más fácilmente. Déjalo funcionando durante unos minutos.\n\n3. Levanta el coche: con la ayuda de un gato o una rampa, levanta el coche para poder acceder al tapón de drenaje del aceite, que se encuentra en la parte inferior del motor.\n\n4. Afloja el tapón de drenaje: con la llave adecuada, afloja el tapón de drenaje y coloca el recipiente debajo para recoger el aceite usado. Deja que el aceite se drene por completo.\n\n5. Cambia el filtro de aceite: una vez que el aceite haya drenado por completo, quita el filtro de aceite antiguo y 

In [23]:
chain.invoke("¿Cómo funciona internamente un catalizador?")



> Entering new MultiPromptChain chain...
mecánica avanzada: {'input': '¿Cómo funciona internamente un catalizador?'}
> Finished chain.


{'input': '¿Cómo funciona internamente un catalizador?',
 'text': 'El catalizador es un componente fundamental en el sistema de escape de un vehículo, ya que su función principal es reducir las emisiones de gases contaminantes al convertirlos en gases menos nocivos. \n\nInternamente, un catalizador está compuesto por un sustrato cerámico o metálico recubierto con metales preciosos como platino, paladio y rodio. Estos metales actúan como catalizadores, acelerando las reacciones químicas que convierten los gases contaminantes en gases menos dañinos.\n\nCuando los gases de escape pasan a través del catalizador, ocurren varias reacciones químicas. En primer lugar, los gases de óxidos de nitrógeno (NOx) se convierten en nitrógeno y oxígeno. Posteriormente, los hidrocarburos sin quemar se oxidan para convertirse en dióxido de carbono y agua. Finalmente, el monóxido de carbono se oxida para convertirse en dióxido de carbono.\n\nEn resumen, el catalizador funciona como un filtro químico que tr